In [13]:
import pandas as pd

### Extract `personalized_test.tsv` of PENS Dataset

In [8]:
personalized_test_df = pd.read_csv("../../data/PENS/personalized_test.tsv", sep="\t")

In [15]:
personalized_test_df.head()

,userid,clicknewsID,posnewID,rewrite_titles
0,NT1,"N108480,N38238,N35068,N110487,N94904,N72378,N4...","N24110,N62769,N36186,N101669,N19241,N72921,N26...",Legal battle looms over Trump EPA's rule chang...
1,NT2,"N34682,N113236,N119039,N90826,N63278,N27346,N5...","N51765,N37815,N109881,N64357,N13381,N45697,N57...",What You Need to Know About GMOs;;What's Up wi...
2,NT3,"N106204,N74279,N55583,N90083,N117690,N91663,N9...","N96078,N11699,N13028,N36049,N87968,N105007,N11...",Don't Know What's Popular This Summer? We've G...
3,NT4,"N61892,N41396,N42145,N24440,N74099,N73577,N123...","N15817,N104663,N10362,N69465,N16287,N70636,N83...",Summer heat putting your pets at risk;;Trip Ad...
4,NT5,"N79801,N52642,N19270,N112075,N37402,N120660,N3...","N61157,N69119,N101472,N122218,N92462,N67440,N5...",Top News Stories from Texas ;;Some Simple Tips...


In [16]:
personalized_test_df['posnewID'] = personalized_test_df['posnewID'].str.split(',')
personalized_test_df['rewrite_titles'] = personalized_test_df['rewrite_titles'].str.split(';;')

personalized_test_df_expanded = personalized_test_df.explode(['posnewID', 'rewrite_titles']).reset_index(drop=True)


### Expanding `personalized_test.tsv`
By Expanding `personalized_test.tsv` will result in multiple summaries, each written by different users, along with their respective history.

In [17]:
personalized_test_df_expanded

,userid,clicknewsID,posnewID,rewrite_titles
0,NT1,"N108480,N38238,N35068,N110487,N94904,N72378,N4...",N24110,Legal battle looms over Trump EPA's rule chang...
1,NT1,"N108480,N38238,N35068,N110487,N94904,N72378,N4...",N62769,Wise choices for stylish updating of old homes
2,NT1,"N108480,N38238,N35068,N110487,N94904,N72378,N4...",N36186,Verlander may be reconsidering his stance on M...
3,NT1,"N108480,N38238,N35068,N110487,N94904,N72378,N4...",N101669,Infamous o.j. Simpson launching official Twitt...
4,NT1,"N108480,N38238,N35068,N110487,N94904,N72378,N4...",N19241,15 year old cori gauff beats Venus Williams at...
...,...,...,...,...
20595,NT103,"N72135,N96473,N68337,N55731,N28068,N122658,N64...",N109539,Maricopa County first death was recorded from ...
20596,NT103,"N72135,N96473,N68337,N55731,N28068,N122658,N64...",N41715,The 2019 Infiniti QX50 shows the future of its...
20597,NT103,"N72135,N96473,N68337,N55731,N28068,N122658,N64...",N12997,Cronos Group may be the deal of a lifetime
20598,NT103,"N72135,N96473,N68337,N55731,N28068,N122658,N64...",N17788,How health insurance companies make psychic he...


### Extract Lookup `news_min.tsv`

In [18]:
news_min_df = pd.read_csv("../../data/PENS/news_min.tsv", sep="\t")

In [19]:
news_min_df

,News ID,Category,Topic,Headline,News body
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec..."
...,...,...,...,...,...
113757,N123757,sports,soccer_fifa_wwc,Hope who? Alyssa Naeher's penalty save sends U...,"LYON, France At the conclusion of the United..."
113758,N123758,sports,baseball_mlb,Chris Sale Explains What Specifically Has Gone...,The first half of Chris Sale's season could be...
113759,N123759,sports,basketball_nba_videos,Raptor fans jam streets to celebrate 1st NBA t...,Canadians are celebrating the country's first ...
113760,N123760,news,newspolitics,Judge won't allow Flynn to fire his attorneys,A federal judge denied the request by Michael ...


### Loading T5 for Doc Embeddings

In [20]:
import re
import string
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords

# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = nltk.WordNetLemmatizer()

# Define a preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    # Join words back into a single string
    return ' '.join(words)

# remove 'NaN' from the `news_min_df`
news_min_df = news_min_df.dropna(subset=['News body'])

# Enable tqdm progress bar for pandas apply
tqdm.pandas()





In [21]:
# Apply preprocessing before generating embeddings
news_min_df['preprocessed_text'] = news_min_df['News body'].progress_apply(preprocess_text)

100%|██████████| 113704/113704 [08:39<00:00, 218.84it/s] 
C:\Users\borad\AppData\Local\Temp\ipykernel_20508\2146980708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_min_df['preprocessed_text'] = news_min_df['News body'].progress_apply(preprocess_text)


In [33]:

# Save to CSV
news_min_df.to_csv('../../data/PENS/preprocessed_news.csv')

In [34]:
news_min_df

,Category,Topic,Headline,News body,preprocessed_text
News ID,,,,,
N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI...",five international allowed count em five first...
N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...,washington senate majority leader mitch mcconn...
N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...,north highland cbs13 fire damaged home north h...
N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...,meghan mccain speaking journalist criticized c...
N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec...",1714 george becomes king georg ludwig elector ...
...,...,...,...,...,...
N123757,sports,soccer_fifa_wwc,Hope who? Alyssa Naeher's penalty save sends U...,"LYON, France At the conclusion of the United...",lyon france conclusion united state womens nat...
N123758,sports,baseball_mlb,Chris Sale Explains What Specifically Has Gone...,The first half of Chris Sale's season could be...,first half chris sale season could described m...
N123759,sports,basketball_nba_videos,Raptor fans jam streets to celebrate 1st NBA t...,Canadians are celebrating the country's first ...,canadian celebrating country first professiona...


### START HERE

In [1]:
import pandas as pd
news_min_df = pd.read_csv('../../data/PENS/preprocessed_news.csv')

In [2]:
news_min_df

,News ID,Category,Topic,Headline,News body,preprocessed_text
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI...",five international allowed count em five first...
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...,washington senate majority leader mitch mcconn...
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...,north highland cbs13 fire damaged home north h...
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...,meghan mccain speaking journalist criticized c...
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec...",1714 george becomes king georg ludwig elector ...
...,...,...,...,...,...,...
113699,N123757,sports,soccer_fifa_wwc,Hope who? Alyssa Naeher's penalty save sends U...,"LYON, France At the conclusion of the United...",lyon france conclusion united state womens nat...
113700,N123758,sports,baseball_mlb,Chris Sale Explains What Specifically Has Gone...,The first half of Chris Sale's season could be...,first half chris sale season could described m...
113701,N123759,sports,basketball_nba_videos,Raptor fans jam streets to celebrate 1st NBA t...,Canadians are celebrating the country's first ...,canadian celebrating country first professiona...
113702,N123760,news,newspolitics,Judge won't allow Flynn to fire his attorneys,A federal judge denied the request by Michael ...,federal judge denied request michael flynns la...


In [4]:
# Set 'News ID' as the index if not already done
news_min_df.set_index('News ID', inplace=True)

# Look up a specific news entry by 'News ID'
news_id = 'N10000'  # Replace with the desired News ID
specific_news = news_min_df.loc[news_id]


print(specific_news)

Category                                                        sports
Topic                                                           soccer
Headline             Predicting Atlanta United's lineup against Col...
News body            Only FIVE internationals allowed, count em, FI...
preprocessed_text    five international allowed count em five first...
Name: N10000, dtype: object


### Loading T5 for Doc Embeddings

In [5]:
import torch

from transformers import T5Tokenizer, T5Model

# Load pre-trained model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5Model.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
def get_embeddings(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=512)
    decoder_input_ids = tokenizer("summarize:", return_tensors="pt").input_ids
    
    with torch.no_grad():
        outputs = model(input_ids=inputs.input_ids, decoder_input_ids=decoder_input_ids, output_hidden_states=True)
        hidden_states = outputs.encoder_hidden_states
        last_layer_hidden_state = hidden_states[-1]
        embeddings = last_layer_hidden_state[:, 0, :]  # First token embedding
    return embeddings.squeeze()

In [9]:
personalized_test_df = pd.read_csv("../../data/PENS/personalized_test.tsv", sep="\t")

### Iterate all Users & Create Sequcne of Embeddings

In [10]:
from tqdm import tqdm
import torch

# Define relation
relations = {
    'click': torch.randn(1, 512),
    'skip': torch.randn(1, 512),
    'gen_summ': torch.randn(1, 512),
    'summ_gen': torch.randn(1, 512)
}

doc_embedding_store = {}
summ_embedding_store = {}

# Iterate all Users & Create Sequence of Embeddings
for index, user in tqdm(personalized_test_df.iterrows(), total=personalized_test_df.shape[0], desc="Processing Users", leave=True):
    
    # Extract and process clicknewsID
    click_list = user["clicknewsID"].split(",")
    
    # Iterate over clicknewsID followed by posnewID
    for doc in tqdm(click_list, desc="Processing Clicked News", leave=False, position=1):
        doc_body = news_min_df.loc[doc]
        if doc not in doc_embedding_store:
            doc_embedding_store[doc] = get_embeddings(doc_body["preprocessed_text"], model, tokenizer).unsqueeze(0)

    # Iterate over rewritten summaries (posnewID)
    for position, summ_doc in enumerate(tqdm(user["posnewID"], desc="Processing Summaries", leave=False, position=2)):
        if summ_doc not in summ_embedding_store:
            summ_embedding_store[summ_doc] = get_embeddings(user["rewrite_titles"][position], model, tokenizer).unsqueeze(0)


Processing Users:   0%|          | 0/103 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [24]:
import torch
from tqdm import tqdm

# Define relations
relations = {
    'click': torch.randn(1, 512),
    'skip': torch.randn(1, 512),
    'gen_summ': torch.randn(1, 512),
    'summ_gen': torch.randn(1, 512)
}

data = []
# Outer loop for processing users
for index, user in tqdm(personalized_test_df.head(2).iterrows(), total=personalized_test_df.shape[0], desc="Processing Users", leave=True):
    print(f"Processing user {index}")
    print(user)

    c_list = user["clicknewsID"].split(",")
    
    # Initialize an empty list for trajectory embeddings
    user_trajectory_embeddings = [] 
    
    # If first user (index 0), concatenate user embedding with the first document
    if index == 0:
        user_embedding = torch.randn(1, 512)  # Example user embedding
        doc_body = news_min_df.loc[c_list[0]]
        
        # Get the document embedding and concatenate with user embedding and 'click' relation
        user_trajectory_embeddings.append(
            torch.cat((user_embedding, relations['click'], get_embeddings(doc_body["preprocessed_text"], model, tokenizer).unsqueeze(0)), dim=0)
        )
    
    # Inner loop for processing clicked news IDs (with tqdm)
    for i in tqdm(range(len(c_list) - 1), desc=f"Processing Clicks for User {index}", leave=False):  # Use len(c_list) - 1 to avoid out of range error
        print(c_list[i])
        doc_body = news_min_df.loc[c_list[i]]
        doc_head = get_embeddings(doc_body["preprocessed_text"], model, tokenizer).unsqueeze(0)
        doc_relation = relations['click']

        # Make sure you have a next document to concatenate
        doc_body = news_min_df.loc[c_list[i + 1]]
        doc_tail = get_embeddings(doc_body["preprocessed_text"], model, tokenizer).unsqueeze(0)

        # Concatenate doc_head, doc_relation, and doc_tail
        user_trajectory_embeddings.append(
            torch.cat((doc_head, doc_relation, doc_tail), dim=0)
        )

    data.append(user_trajectory_embeddings)
# Now `user_trajectory_embeddings` will contain concatenated embeddings for each user


Processing Users:   0%|          | 0/103 [00:00<?, ?it/s]

Processing user 0
userid                                                          NT1
clicknewsID       N108480,N38238,N35068,N110487,N94904,N72378,N4...
posnewID          N24110,N62769,N36186,N101669,N19241,N72921,N26...
rewrite_titles    Legal battle looms over Trump EPA's rule chang...
Name: 0, dtype: object


N108480


N38238


N35068


...

N14201


N48780


N53338


N105543


Processing Users:   2%|▏         | 2/103 [03:35<3:01:45, 107.97s/it]


In [28]:
user_trajectory_embeddings[1].shape

torch.Size([3, 512])

In [18]:
user_embedding = torch.randn(1, 512)
torch.cat((user_embedding, user_embedding + 1, user_embedding + 2), dim=0).shape

torch.Size([3, 512])

In [16]:
user_embedding.shape

torch.Size([1, 512])